#  Inférence de l'effet - Stratégie Multilabels - Approche deep Learning
Dans ce Notebook, nous cosntruisons un modèle qui permet d'inférer l'EFFET à partir de la classification de l'incident et des données textuelles en ce basant sur des reseau récurents commes GRU/LSTM etc.

En effet, ces approches ont montré des réultats très encourageant et nous voulons explorer cette direction pour peut être faire des réceau recurent notre modèle par défault.

Nous considérons ce problème comme un problème de classification multiclasses et multilabels. En effet, il y a plusieurs effets possibles et un incidents peut entrainer plusieurs effets.

Dans ce note book nous nous posons les questions suivantes : 
- Quel est l'impact du drop out ?
- Rajouter des couches augmentent-ils les performaces ?
- L'utilisation de réseaux bidirectionnel est-elle pertinente ?
- Une couche d'attention est-elle utile ?
- Attention is all we need, really ?
- Utilisation des embeddings 
- Concaténation des modèles sur différentes entrées ?

In [8]:
import pandas as pd
import numpy as np
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding,SpatialDropout1D, Bidirectional,SimpleRNN,Input, concatenate, Reshape
import tensorflow 

from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from keras.layers import Concatenate, GlobalMaxPool1D, Dropout
from tensorflow.keras.layers import Attention

from sklearn.metrics import  precision_score, recall_score, f1_score


tensorflow.random.set_seed(1234)

## 1. Chargement des données

In [4]:
mlb = MultiLabelBinarizer()

train = pd.read_pickle('./data_split/train.pkl')
# Pour faire un modèle sans le 
#train = train[~train['TEF_ID'].map(lambda x : 106 in x)]
X_train = train[['FABRICANT','CLASSIFICATION','DESCRIPTION_INCIDENT','ETAT_PATIENT']]
y_train = mlb.fit_transform(train['TDY_ID'])
test =  pd.read_pickle('./data_split/test.pkl')
#test = test[~test['TEF_ID'].map(lambda x : k in x)]
X_test = test[['FABRICANT','CLASSIFICATION','DESCRIPTION_INCIDENT','ETAT_PATIENT']]
y_test = mlb.transform(test['TDY_ID'])


X_train_dgs = np.load('results/dgs_camenbert_train_vec.npy')
X_test_dgs =np.load('results/dgs_camenbert_test_vec.npy')



df_effets = pd.read_csv("data/ref_MRV/referentiel_dispositif_effets_connus.csv",delimiter=';',encoding='ISO-8859-1')
df_dys = pd.read_csv("data/ref_MRV/referentiel_dispositif_dysfonctionnement.csv",delimiter=';',encoding='ISO-8859-1')

/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:995: UserWarning: unknown class(es) [175, 220, 229, 23, 254, 277, 278, 373, 440, 446, 465, 541, 564, 566, 572, 581, 583, 588, 592, 594, 613, 618] will be ignored
  .format(sorted(unknown, key=str)))


## 2.1 LSTM et TFIDF, une première baseline

In [5]:
%%time
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer
import spacy
nlp =spacy.load('fr')
from spacy.lang.fr.stop_words import STOP_WORDS

preprocess = ColumnTransformer(
    [('description_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 10000,norm = 'l2'), 'DESCRIPTION_INCIDENT'),
     
     ('etat_pat_tfidf', TfidfVectorizer(sublinear_tf=True, min_df=3,ngram_range=(1, 1),
                                       stop_words=STOP_WORDS,
                                       max_features = 10000,norm = 'l2'), 'ETAT_PATIENT'),
     
     ('fabricant_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 5000,norm = 'l2'), 'FABRICANT')
     ],
    
    remainder='passthrough')

X_train_, X_test_ =preprocess.fit_transform(X_train),preprocess.transform(X_test)

from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=1000)
X_train_ = svd.fit_transform(X_train_)
X_test_ = svd.transform(X_test_)


/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['qu', 'quelqu'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['qu', 'quelqu'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['qu', 'quelqu'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


CPU times: user 3min 4s, sys: 2min 52s, total: 5min 57s
Wall time: 1min 13s


In [6]:
X_train_ = np.reshape(X_train_, (X_train_.shape[0], 1, X_train_.shape[1]))
X_test_ = np.reshape(X_test_, (X_test_.shape[0], 1, X_test_.shape[1]))

In [10]:
model = Sequential()
model.add(LSTM(200))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

epochs = 10
batch_size = 32

history = model.fit(X_train_, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

score,cat_acc = model.evaluate(X_test_,y_test)

y_pred = model.predict(X_test_)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Train on 21059 samples, validate on 5265 samples
Epoch 1/10
21059/21059 [==============================] - 10s 470us/step - loss: 0.0103 - categorical_accuracy: 0.2212 - val_loss: 0.0095 - val_categorical_accuracy: 0.2577
Epoch 2/10
21059/21059 [==============================] - 11s 520us/step - loss: 0.0082 - categorical_accuracy: 0.3351 - val_loss: 0.0084 - val_categorical_accuracy: 0.3077
Epoch 3/10
21059/21059 [==============================] - 12s 569us/step - loss: 0.0069 - categorical_accuracy: 0.4104 - val_loss: 0.0075 - val_categorical_accuracy: 0.3510
Epoch 4/10
21059/21059 [==============================] - 12s 547us/step - loss: 0.0059 - categorical_accuracy: 0.4705 - val_loss: 0.0070 - val_categorical_accuracy: 0.3787
Epoch 5/10
21059/21059 [==============================] - 11s 537us/step - loss: 0.0052 - categorical_accuracy: 0.5120 - val_loss: 0.0067 - val_categorical_accuracy: 0.3951
Epoch 6/10
21059/21059 [==============================] - 11s 513us/step - loss: 0.004

## Commentaires : 

Nous obtenons un score de base line de 0.6730. Nous allons essayer d'améliorer ce score avec difféntes approches : 
- Remplacer le tfidf par un embedding entrainé par le modèle 
    - Embedding simple
    - Embedding par colonnes
    
- Essayer un modèle biLSTM ou GRU ou bi GRU
- Essayer de rajouter une couche d'attention dans Keras


In [11]:
model = Sequential()
model.add(LSTM(200, dropout=0.15))
model.add(Dense(y_train.shape[1], activation='softmax'))



model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['categorical_accuracy'])

epochs = 4
batch_size = 32

history = model.fit(X_train_, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

score,cat_acc = model.evaluate(X_test_,y_test)

y_pred = model.predict(X_test_)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Train on 21059 samples, validate on 5265 samples
Epoch 1/4
21059/21059 [==============================] - 17s 812us/step - loss: 0.0093 - categorical_accuracy: 0.2753 - val_loss: 0.0084 - val_categorical_accuracy: 0.3102
Epoch 2/4
21059/21059 [==============================] - 14s 664us/step - loss: 0.0069 - categorical_accuracy: 0.4050 - val_loss: 0.0070 - val_categorical_accuracy: 0.3789
Epoch 3/4
21059/21059 [==============================] - 17s 792us/step - loss: 0.0056 - categorical_accuracy: 0.4744 - val_loss: 0.0066 - val_categorical_accuracy: 0.3964
Epoch 4/4
6580/6580 [==============================] - 2s 255us/step
loss :  0.005549738023396139
categorical accuracy:  0.46702128648757935
####################################
For threshold:  0.01
Samples-average quality numbers
Precision: 0.2107, Recall: 0.8676, F1-measure: 0.2918
For threshold:  0.04
Samples-average quality numbers
Precision: 0.3481, Recall: 0.7245, F1-measure: 0.4276
For threshold:  0.06
Samples-average qualit

# 2.2 Comment performe un reseau de BiLSTM  ?


In [12]:
from keras.layers import Bidirectional

In [13]:
model = Sequential()
model.add(Bidirectional(LSTM(200, dropout=0.1)))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

epochs = 5
batch_size = 32

history = model.fit(X_train_, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

score,cat_acc = model.evaluate(X_test_,y_test)

y_pred = model.predict(X_test_)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Train on 21059 samples, validate on 5265 samples
Epoch 1/5
21059/21059 [==============================] - 25s 1ms/step - loss: 0.0100 - categorical_accuracy: 0.2330 - val_loss: 0.0092 - val_categorical_accuracy: 0.2705
Epoch 2/5
21059/21059 [==============================] - 23s 1ms/step - loss: 0.0078 - categorical_accuracy: 0.3578 - val_loss: 0.0079 - val_categorical_accuracy: 0.3273
Epoch 3/5
21059/21059 [==============================] - 22s 1ms/step - loss: 0.0065 - categorical_accuracy: 0.4308 - val_loss: 0.0071 - val_categorical_accuracy: 0.3719
Epoch 4/5
21059/21059 [==============================] - 17s 813us/step - loss: 0.0055 - categorical_accuracy: 0.4848 - val_loss: 0.0067 - val_categorical_accuracy: 0.3911
Epoch 5/5
6580/6580 [==============================] - 2s 329us/step
loss :  0.00564778517014531
categorical accuracy:  0.4729483425617218
####################################
For threshold:  0.01
Samples-average quality numbers
Precision: 0.2043, Recall: 0.8603, F1-me

## Commentaire ?

L'augementation de la performance est très minime : 0.6733 ?
Nous remarquons que notre modèle sur apprend assez rapidement, nous allons essayer d'ajouter une couche de dropout pour limiter ce sur apprentissage.
## Si on ajoute du dropout ?

In [14]:
model = Sequential()
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.1)))
model.add(GlobalMaxPool1D())
model.add(Dense(100, activation="elu"))
model.add(Dropout(0.1))
model.add( Dense(y_train.shape[1], activation="softmax"))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])


epochs = 10
batch_size = 32

history = model.fit(X_train_, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

score,cat_acc = model.evaluate(X_test_,y_test)

y_pred = model.predict(X_test_)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Train on 21059 samples, validate on 5265 samples
Epoch 1/10
21059/21059 [==============================] - 25s 1ms/step - loss: 0.0096 - categorical_accuracy: 0.2406 - val_loss: 0.0090 - val_categorical_accuracy: 0.2651
Epoch 2/10
21059/21059 [==============================] - 22s 1ms/step - loss: 0.0077 - categorical_accuracy: 0.3513 - val_loss: 0.0079 - val_categorical_accuracy: 0.3206
Epoch 3/10
21059/21059 [==============================] - 22s 1ms/step - loss: 0.0066 - categorical_accuracy: 0.4090 - val_loss: 0.0073 - val_categorical_accuracy: 0.3499
Epoch 4/10
21059/21059 [==============================] - 21s 1ms/step - loss: 0.0060 - categorical_accuracy: 0.4440 - val_loss: 0.0069 - val_categorical_accuracy: 0.3742
Epoch 5/10
21059/21059 [==============================] - 21s 1ms/step - loss: 0.0055 - categorical_accuracy: 0.4749 - val_loss: 0.0067 - val_categorical_accuracy: 0.3871
Epoch 6/10
21059/21059 [==============================] - 19s 900us/step - loss: 0.0051 - catego

## Commentaire ?

Les performances changent très peu,  nous allons essayer d'augementer la profondeur de notre reseau.

## Si on ajoute plusieurs couches ?

In [15]:
from tensorflow.keras import regularizers
reg = regularizers.l2(1e-4)
model = Sequential()
model.add(Bidirectional(LSTM(256,  return_sequences=True, dropout=0.1)))
model.add((SimpleRNN(128,return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.1))
model.add(Dense(y_train.shape[1], activation="softmax"))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])


epochs = 10
batch_size = 32

history = model.fit(X_train_, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

score,cat_acc = model.evaluate(X_test_,y_test)

y_pred = model.predict(X_test_)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Train on 21059 samples, validate on 5265 samples
Epoch 1/10
21059/21059 [==============================] - 31s 1ms/step - loss: 0.0095 - categorical_accuracy: 0.2496 - val_loss: 0.0089 - val_categorical_accuracy: 0.2832
Epoch 2/10
21059/21059 [==============================] - 25s 1ms/step - loss: 0.0075 - categorical_accuracy: 0.3683 - val_loss: 0.0077 - val_categorical_accuracy: 0.3328
Epoch 3/10
21059/21059 [==============================] - 26s 1ms/step - loss: 0.0064 - categorical_accuracy: 0.4279 - val_loss: 0.0071 - val_categorical_accuracy: 0.3654
Epoch 4/10
21059/21059 [==============================] - 21s 1ms/step - loss: 0.0057 - categorical_accuracy: 0.4646 - val_loss: 0.0068 - val_categorical_accuracy: 0.3869
Epoch 5/10
21059/21059 [==============================] - 24s 1ms/step - loss: 0.0053 - categorical_accuracy: 0.4910 - val_loss: 0.0067 - val_categorical_accuracy: 0.3892
Epoch 6/10
21059/21059 [==============================] - 22s 1ms/step - loss: 0.0050 - categori

/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Samples-average quality numbers
Precision: 0.2242, Recall: 0.8423, F1-measure: 0.3096
For threshold:  0.04
Samples-average quality numbers
Precision: 0.3604, Recall: 0.7147, F1-measure: 0.4368
For threshold:  0.06


/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Samples-average quality numbers
Precision: 0.4026, Recall: 0.6706, F1-measure: 0.4675
For threshold:  0.08
Samples-average quality numbers
Precision: 0.4274, Recall: 0.6379, F1-measure: 0.4820
For threshold:  0.1
Samples-average quality numbers
Precision: 0.4460, Recall: 0.6097, F1-measure: 0.4897
For threshold:  0.12
Samples-average quality numbers
Precision: 0.4597, Recall: 0.5851, F1-measure: 0.4931
For threshold:  0.14
Samples-average quality numbers
Precision: 0.4677, Recall: 0.5606, F1-measure: 0.4914
For threshold:  0.16
Samples-average quality numbers
Precision: 0.4719, Recall: 0.5412, F1-measure: 0.4877
For threshold:  0.2


/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


Samples-average quality numbers
Precision: 0.4770, Recall: 0.5113, F1-measure: 0.4805
For threshold:  0.25
Samples-average quality numbers
Precision: 0.4709, Recall: 0.4773, F1-measure: 0.4647
For threshold:  0.3
Samples-average quality numbers
Precision: 0.4571, Recall: 0.4485, F1-measure: 0.4458
For threshold:  0.35
Samples-average quality numbers
Precision: 0.4375, Recall: 0.4211, F1-measure: 0.4236
For threshold:  0.4
Samples-average quality numbers
Precision: 0.4177, Recall: 0.3962, F1-measure: 0.4023
For threshold:  0.5
Samples-average quality numbers
Precision: 0.3723, Recall: 0.3537, F1-measure: 0.3596
For threshold:  0.6
Samples-average quality numbers
Precision: 0.3266, Recall: 0.3120, F1-measure: 0.3166
For threshold:  0.7
Samples-average quality numbers
Precision: 0.2781, Recall: 0.2691, F1-measure: 0.2719


### Commentaires : 

Encore une fois, le modèle semble avoir atteint un plafond de verre qui ne soit pas dépassable ni par la prodondeur, ni par l'ajout de droptout. Nous allons essayer d'ajouter une couche d'attention.

## Une couche d'attention ?

In [17]:
from keras_self_attention import SeqSelfAttention
import keras
model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(keras.layers.Flatten(name='Flatten'))
model.add(Dense(y_train.shape[1], activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])


epochs = 10
batch_size = 32

history = model.fit(X_train_, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

score,cat_acc = model.evaluate(X_test_,y_test)

y_pred = model.predict(X_test_)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.26,0.27,0.28,0.29,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Train on 21059 samples, validate on 5265 samples
Epoch 1/10
21059/21059 [==============================] - 17s 806us/step - loss: 0.1038 - categorical_accuracy: 0.1660 - val_loss: 0.0116 - val_categorical_accuracy: 0.1896
Epoch 2/10
21059/21059 [==============================] - 15s 701us/step - loss: 0.0104 - categorical_accuracy: 0.1873 - val_loss: 0.0106 - val_categorical_accuracy: 0.1896
Epoch 3/10
21059/21059 [==============================] - 14s 653us/step - loss: 0.0099 - categorical_accuracy: 0.1999 - val_loss: 0.0102 - val_categorical_accuracy: 0.2053
Epoch 4/10
21059/21059 [==============================] - 17s 793us/step - loss: 0.0094 - categorical_accuracy: 0.2421 - val_loss: 0.0099 - val_categorical_accuracy: 0.2251
Epoch 5/10
21059/21059 [==============================] - 16s 765us/step - loss: 0.0090 - categorical_accuracy: 0.2904 - val_loss: 0.0096 - val_categorical_accuracy: 0.2574
Epoch 6/10
21059/21059 [==============================] - 16s 767us/step - loss: 0.008

/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Samples-average quality numbers
Precision: 0.1190, Recall: 0.7778, F1-measure: 0.1915
For threshold:  0.04


/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Samples-average quality numbers
Precision: 0.2654, Recall: 0.5994, F1-measure: 0.3397
For threshold:  0.06
Samples-average quality numbers
Precision: 0.3179, Recall: 0.5383, F1-measure: 0.3727
For threshold:  0.08


/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


Samples-average quality numbers
Precision: 0.3401, Recall: 0.4933, F1-measure: 0.3796
For threshold:  0.1
Samples-average quality numbers
Precision: 0.3529, Recall: 0.4598, F1-measure: 0.3798
For threshold:  0.12
Samples-average quality numbers
Precision: 0.3520, Recall: 0.4315, F1-measure: 0.3713
For threshold:  0.14
Samples-average quality numbers
Precision: 0.3498, Recall: 0.4107, F1-measure: 0.3635
For threshold:  0.16
Samples-average quality numbers
Precision: 0.3476, Recall: 0.3930, F1-measure: 0.3569
For threshold:  0.2
Samples-average quality numbers
Precision: 0.3395, Recall: 0.3666, F1-measure: 0.3431
For threshold:  0.25
Samples-average quality numbers
Precision: 0.3286, Recall: 0.3412, F1-measure: 0.3279
For threshold:  0.26
Samples-average quality numbers
Precision: 0.3255, Recall: 0.3363, F1-measure: 0.3242
For threshold:  0.27
Samples-average quality numbers
Precision: 0.3235, Recall: 0.3324, F1-measure: 0.3218
For threshold:  0.28
Samples-average quality numbers
Precisi

## Commentaire : 
Les résultats sembles similaire. Toutefois, la loss de validation est bien meilleur sans pour autant augmenter le f1 score. Nous allons essayer de faire seulement un réseau basé sur l'attention. Nous utilison ici, une verion multiplicative de l'attention.

In [18]:
import keras
from keras_self_attention import SeqSelfAttention

inputs = keras.layers.Input(shape=(1,1000))
#lstm = keras.layers.Bidirectional(keras.layers.LSTM(units=256,
                                                    #return_sequences=True))(inputs)
att = SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
                       kernel_regularizer=keras.regularizers.l2(1e-4),
                       bias_regularizer=keras.regularizers.l1(1e-4),
                       attention_regularizer_weight=1e-4,
                       name='Attention')(inputs)

flatten = keras.layers.Flatten(name='Flatten')(att)
dense = keras.layers.Dense(units=y_train.shape[1], name='Dense')(flatten)

model = keras.models.Model(inputs=inputs, outputs=[dense])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])


epochs = 20
batch_size = 32

history = model.fit(X_train_, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

score,cat_acc = model.evaluate(X_test_,y_test)

y_pred = model.predict(X_test_)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Train on 21059 samples, validate on 5265 samples
Epoch 1/20
21059/21059 [==============================] - 18s 831us/step - loss: 0.0192 - categorical_accuracy: 0.2267 - val_loss: 0.0149 - val_categorical_accuracy: 0.2500
Epoch 2/20
21059/21059 [==============================] - 18s 844us/step - loss: 0.0127 - categorical_accuracy: 0.3561 - val_loss: 0.0139 - val_categorical_accuracy: 0.2976
Epoch 3/20
21059/21059 [==============================] - 18s 834us/step - loss: 0.0119 - categorical_accuracy: 0.4041 - val_loss: 0.0136 - val_categorical_accuracy: 0.3151
Epoch 4/20
21059/21059 [==============================] - 15s 722us/step - loss: 0.0113 - categorical_accuracy: 0.4303 - val_loss: 0.0135 - val_categorical_accuracy: 0.3303
Epoch 5/20
21059/21059 [==============================] - 13s 597us/step - loss: 0.0110 - categorical_accuracy: 0.4531 - val_loss: 0.0134 - val_categorical_accuracy: 0.3371
Epoch 6/20
21059/21059 [==============================] - 15s 731us/step - loss: 0.010

/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Samples-average quality numbers
Precision: 0.2380, Recall: 0.6459, F1-measure: 0.3210
For threshold:  0.04
Samples-average quality numbers
Precision: 0.2799, Recall: 0.6191, F1-measure: 0.3582
For threshold:  0.06


/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


Samples-average quality numbers
Precision: 0.3050, Recall: 0.5986, F1-measure: 0.3772
For threshold:  0.08
Samples-average quality numbers
Precision: 0.3287, Recall: 0.5798, F1-measure: 0.3936
For threshold:  0.1
Samples-average quality numbers
Precision: 0.3483, Recall: 0.5579, F1-measure: 0.4040
For threshold:  0.12
Samples-average quality numbers
Precision: 0.3625, Recall: 0.5363, F1-measure: 0.4097
For threshold:  0.14
Samples-average quality numbers
Precision: 0.3767, Recall: 0.5146, F1-measure: 0.4140
For threshold:  0.16
Samples-average quality numbers
Precision: 0.3848, Recall: 0.4945, F1-measure: 0.4139
For threshold:  0.2
Samples-average quality numbers
Precision: 0.3857, Recall: 0.4539, F1-measure: 0.4025
For threshold:  0.25
Samples-average quality numbers
Precision: 0.3708, Recall: 0.4057, F1-measure: 0.3774
For threshold:  0.3
Samples-average quality numbers
Precision: 0.3503, Recall: 0.3649, F1-measure: 0.3503
For threshold:  0.35
Samples-average quality numbers
Precisio

### Commentaire : 

Le modèle semble réellement bloqué a une valeure seuil de 0.67. Que ce soit des réseau recurent ou bien des mécanismes d'attention le score reste identique après 10 epochs. Nous allons donc changer de représentation car nous pensons avoir atteint les limite de la représentation en tfidf+SVD.

## Et si l'on changait de représentation ? Utilisation d'Embedding globaux

In [19]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [20]:
df_train = train['DESCRIPTION_INCIDENT']+'. '+train['ETAT_PATIENT']#+ '. '+train['ACTION_PATIENT']
df_test = test['DESCRIPTION_INCIDENT']+'. '+test['ETAT_PATIENT']#+'. '+test['ACTION_PATIENT']

In [21]:
# The maximum number of words to be used. 
MAX_NB_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 300
# This is fixed.
EMBEDDING_DIM = 300

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(df_train.values)
word_index = tokenizer.word_index
print(len(word_index))

X_train = tokenizer.texts_to_sequences(df_train.values)
X_test = tokenizer.texts_to_sequences(df_test.values)
word2index_inputs =  tokenizer.word_index

X_train = pad_sequences(X_train,MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test,MAX_SEQUENCE_LENGTH)
print(X_train.shape,X_test.shape)

61213
(26324, 300) (6580, 300)


In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(200))
model.add(Dense(y_train.shape[1],activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

epochs = 10
batch_size = 64

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)



y_pred = model.predict(X_test)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

/home/user1/DGS-env/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 21059 samples, validate on 5265 samples
Epoch 1/10
21059/21059 [==============================] - 826s 39ms/step - loss: 0.0103 - categorical_accuracy: 0.1853 - val_loss: 0.0104 - val_categorical_accuracy: 0.1896
Epoch 2/10
21059/21059 [==============================] - 855s 41ms/step - loss: 0.0096 - categorical_accuracy: 0.1999 - val_loss: 0.0099 - val_categorical_accuracy: 0.1987
Epoch 3/10
21059/21059 [==============================] - 845s 40ms/step - loss: 0.0091 - categorical_accuracy: 0.2434 - val_loss: 0.0098 - val_categorical_accuracy: 0.1937
Epoch 4/10
21059/21059 [==============================] - 827s 39ms/step - loss: 0.0087 - categorical_accuracy: 0.2716 - val_loss: 0.0095 - val_categorical_accuracy: 0.2133
Epoch 5/10
21056/21059 [============================>.] - ETA: 0s - loss: 0.0082 - categorical_accuracy: 0.3124

### Commentaire : 
Les embedding appris n'offre pas de meilleurs résultats pour l'instant. Nous allons essayer de créer des embeddings par colonnes de texte, cette solution c'etait avérée fructueuse dans le cas du tfidf.

## Et si l'on séparait les entrées ?

In [ ]:
# Model constants.
EMBEDDING_DIM =300
MAX_SEQUENCE_LENGTH =300
MAX_NB_WORDS = 50000

def vectorize(df_train,df_test,MAX_NB_WORDS,MAX_SEQUENCE_LENGTH ):
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(df_train.values)
    word_index = tokenizer.word_index
    print(len(word_index))

    X_train = tokenizer.texts_to_sequences(df_train.values)
    X_test = tokenizer.texts_to_sequences(df_test.values)
    word2index_inputs =  tokenizer.word_index

    X_train = pad_sequences(X_train,MAX_SEQUENCE_LENGTH)
    X_test = pad_sequences(X_test,MAX_SEQUENCE_LENGTH)
    return (X_train, X_test)
TRAIN = []
for col in ['DESCRIPTION_INCIDENT', 'ETAT_PATIENT', 'FABRICANT'] : 
    X_train,X_test = vectorize(train[col],test[col],MAX_NB_WORDS,MAX_SEQUENCE_LENGTH )
    TRAIN.append((X_train,X_test))

In [ ]:
inputs_1 = Input(shape=(MAX_SEQUENCE_LENGTH,))
inputs_2 = Input(shape=(MAX_SEQUENCE_LENGTH,))
inputs_3 = Input(shape=(MAX_SEQUENCE_LENGTH,))
#x = Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputs)
#x = Reshape((MAX_SEQUENCE_LENGTH,1,))(inputs_1)
x = Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputs_1)
x = SpatialDropout1D(0.2)(x)
x = LSTM(200)(x)


#y = Reshape((MAX_SEQUENCE_LENGTH,1,))(inputs_2)
y = Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputs_2)
y = SpatialDropout1D(0.2)(y)
y = LSTM(200)(y)


#z = Reshape((MAX_SEQUENCE_LENGTH,1,))(inputs_3)
z = Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputs_3)
z = SpatialDropout1D(0.2)(z)
z = LSTM(200)(z)


w = concatenate([x, y, z])

out =  Dense(y_train.shape[1],activation='softmax')(w)

model = keras.models.Model(inputs=[inputs_1,inputs_2,inputs_3], outputs=out)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

model.summary()


In [ ]:
history = model.fit([TRAIN[0][0],TRAIN[1][0],TRAIN[2][0]], y_train, epochs=10, validation_split=0.2, verbose=1, batch_size = 64)

In [ ]:
y_pred = model.predict([TRAIN[0][1],TRAIN[1][1],TRAIN[2][1]])

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

# Commentaire :
Nous observons que notre modèle sur apprend très rapidement et de manière importante. nous avons deux solutions classiques pour contrer cet effet : 
- Regularisation
- Drop Out
- Netoyer les données avec clean text

In [ ]:
import spacy
nlp =spacy.load('fr')
from spacy.lang.fr.stop_words import STOP_WORDS

import clean_text
stop_words = STOP_WORDS


def vectorize(df_train,df_test,MAX_NB_WORDS,MAX_SEQUENCE_LENGTH,stopword = False ):
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    if stopword==True : 
        df_train = df_train.map(lambda x: clean_text.preprocess_text(x))
        df_train = df_train.map(lambda x: [item for item in x.split(" ") if item not in stop_words])
    
    tokenizer.fit_on_texts(df_train.values)
    word_index = tokenizer.word_index
    print(len(word_index))

    X_train = tokenizer.texts_to_sequences(df_train.values)
    X_test = tokenizer.texts_to_sequences(df_test.values)
    word2index_inputs =  tokenizer.word_index

    X_train = pad_sequences(X_train,MAX_SEQUENCE_LENGTH)
    X_test = pad_sequences(X_test,MAX_SEQUENCE_LENGTH)
    return (X_train, X_test)
TRAIN = []
for col in ['DESCRIPTION_INCIDENT', 'ETAT_PATIENT', 'FABRICANT'] : 
    X_train,X_test = vectorize(train[col],test[col],MAX_NB_WORDS,MAX_SEQUENCE_LENGTH,stopword=True )
    TRAIN.append((X_train,X_test))

In [ ]:
inputs_1 = Input(shape=(MAX_SEQUENCE_LENGTH,))
inputs_2 = Input(shape=(MAX_SEQUENCE_LENGTH,))
inputs_3 = Input(shape=(MAX_SEQUENCE_LENGTH,))
#x = Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputs)
#x = Reshape((MAX_SEQUENCE_LENGTH,1,))(inputs_1)
x = Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputs_1)
x = SpatialDropout1D(0.2)(x)
x = LSTM(200)(x)


#y = Reshape((MAX_SEQUENCE_LENGTH,1,))(inputs_2)
y = Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputs_2)
y = SpatialDropout1D(0.2)(y)
y = LSTM(200)(y)


#z = Reshape((MAX_SEQUENCE_LENGTH,1,))(inputs_3)
z = Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputs_3)
z = SpatialDropout1D(0.2)(z)
z = LSTM(200)(z)


w = concatenate([x, y, z])

out =  Dense(y_train.shape[1],activation='softmax')(w)

model = keras.models.Model(inputs=[inputs_1,inputs_2,inputs_3], outputs=out)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

print(model.summary())

In [ ]:
history = model.fit([TRAIN[0][0],TRAIN[1][0],TRAIN[2][0]], y_train, epochs=5, validation_split=0.2, verbose=1, batch_size = 64)

y_pred = model.predict([TRAIN[0][1],TRAIN[1][1],TRAIN[2][1]])

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

# Conclusion : 
 Suite aux diverses expériences que nousa vosn mener dans ce notebook, nos conclusions sont : 
- La tf-idf est une représentation riche qui est difficile à battre tant en termes de performances qu'en temps de calcul.
- Les modèles d'attention régularisés évitent mieux le sur-apprentissage que les modèles recurent avec du drop out
- séparer les collones pour encoder offre de meillers résultats mais toujours moins bon que ce proposé par le TF-iDF
- Lef ait d'enlever les stop_words n'est pas 


Ce que nous devons essayer : 
- Multi head attention https://www.kaggle.com/fareise/multi-head-self-attention-for-text-classification, https://github.com/CyberZHG/keras-multi-head
- Hierarchical attention : https://paperswithcode.com/paper/hierarchical-attentional-hybrid-neural
- Concatenation des embedings et du tfidf
- chercher de nouvelles méthodes de régularisation pour les réseaux récurrents
- tester les CNN : https://www.kaggle.com/sanikamal/text-classification-with-python-and-keras
- ajouter une couche de positinal encoding : https://github.com/kaushalshetty/Positional-Encoding
- Librairie à essyaer rapidement :
    - text-classification-keras : https://pypi.org/project/text-classification-keras/
    - pytext :  https://github.com/facebookresearch/pytext

- approche avec des emmbedings déjà entrainés : 
    - https://adventuresinmachinelearning.com/word2vec-keras-tutorial/
    - https://medium.com/@ppasumarthi_69210/word-embeddings-in-keras-be6bb3092831
    